In [1]:
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime, timedelta
import numpy as np

# Fetch SPY data from yfinance
spy = yf.Ticker("SPY")
spy_data = spy.history(period="max")

# Ensure datetime objects are tz-naive for compatibility
start = spy_data.index.min().tz_localize(None)
end = datetime.now()

# Fetch Interest Rate data from FRED
interest_rate_data = pdr.get_data_fred('FEDFUNDS', start, end)

# Merging datasets
spy_data.index = spy_data.index.tz_localize(None)
interest_rate_data.index = interest_rate_data.index.tz_localize(None)
combined_data = pd.merge(spy_data, interest_rate_data, how='inner', left_index=True, right_index=True)

# Defining windows for short-term and long-term analysis
window_short_term = 30  # Short-term window (e.g., 30 days)
window_long_term = 365  # Long-term window (e.g., 1 year)

# Function to analyze effects around rate change dates
def analyze_rate_change_effects(rate_change_date, window_short, window_long):
    short_term_end = rate_change_date + timedelta(days=window_short)
    long_term_end = rate_change_date + timedelta(days=window_long)

    short_term_data = combined_data[(combined_data.index >= rate_change_date) & (combined_data.index <= short_term_end)]
    long_term_data = combined_data[(combined_data.index >= rate_change_date) & (combined_data.index <= long_term_end)]

    # Using head(1) and tail(1) to safely access the first and last rows
    if not short_term_data.empty and not long_term_data.empty:
        short_term_start_price = short_term_data['Close'].head(1).item()
        short_term_end_price = short_term_data['Close'].tail(1).item()
        long_term_start_price = long_term_data['Close'].head(1).item()
        long_term_end_price = long_term_data['Close'].tail(1).item()

        short_term_return = (short_term_end_price / short_term_start_price) - 1
        long_term_return = (long_term_end_price / long_term_start_price) - 1
    else:
        short_term_return = np.nan
        long_term_return = np.nan

    return short_term_return, long_term_return

# Analyzing effects for each significant rate change
significant_rate_changes = interest_rate_data['FEDFUNDS'].diff().dropna().abs().nlargest(10).index
results = {}
for date in significant_rate_changes:
    results[date] = analyze_rate_change_effects(date, window_short_term, window_long_term)

# Outputting results
for date, (short_term, long_term) in results.items():
    print(f"Rate Change Date: {date.strftime('%Y-%m-%d')}, Short-term Return: {short_term:.2%}, Long-term Return: {long_term:.2%}")

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


Rate Change Date: 2008-02-01, Short-term Return: 0.00%, Long-term Return: -40.24%
Rate Change Date: 2020-03-01, Short-term Return: nan%, Long-term Return: nan%
Rate Change Date: 2008-10-01, Short-term Return: 0.00%, Long-term Return: -8.98%
Rate Change Date: 2022-11-01, Short-term Return: 5.95%, Long-term Return: 11.65%
Rate Change Date: 2022-08-01, Short-term Return: 0.00%, Long-term Return: 12.94%
Rate Change Date: 2020-04-01, Short-term Return: 14.89%, Long-term Return: 65.36%
Rate Change Date: 2001-05-01, Short-term Return: 0.00%, Long-term Return: -12.95%
Rate Change Date: 2001-09-01, Short-term Return: 0.00%, Long-term Return: -14.02%
Rate Change Date: 2008-11-01, Short-term Return: 0.00%, Long-term Return: 28.66%
Rate Change Date: 2001-10-01, Short-term Return: 0.00%, Long-term Return: -16.61%
